# Final work

# Setup the learning environment

To begin, we import some library modules and functions that we will use.   


In [1]:
import cv2
import imutils
from imutils import perspective
from imutils import contours
from scipy.spatial import distance as dist
import numpy as np
import argparse
import pickle
import time
from math import sqrt
from matplotlib import image
from matplotlib import pyplot as plt
from rembg import remove
from tkinter import *
from tkinter import ttk, filedialog
from tkinter.filedialog import askopenfile
import os


In [2]:
def midpoint(ptA, ptB):
	return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)
#def mesureheight(image);

In [3]:
alg = "haarcascade_frontalface_default.xml"
haar_cascade = cv2.CascadeClassifier(alg)

In [4]:
def detect(frame):
    grayImg = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    face = haar_cascade.detectMultiScale(grayImg,1.3,4)


    # getting no. of human detected
    print('Human Detected : ', len(face))
    height = 0

    # loop over all detected humans
    for (x, y, w, h) in face:
        pad_w, pad_h = int(0.5 * w), int(0.1 * h)
        #cv2.rectangle(image, (x - pad_w, y - pad_h), (x + w + pad_w, 750 + y + h + pad_h), (0, 255, 0), 20)
        pt1 = (12,y - pad_h)
        pt2 = (12,700 + y + h - pad_h)
        color = (255, 0, 0)

        thickness=20


        #pt1 = (((x + pad_w) + (x + w - pad_w))/2, y + pad_h)
        #pt2 = (x + w - pad_w, y + h - pad_h) 
        cv2.line(frame, pt1, pt2,color,thickness)

        # convert it to grayscale, and blur it slightly
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (3, 3), 0)
        # perform edge detection, then perform a dilation + erosion to
        # close gaps in between object edges
        edged = cv2.Canny(gray, 5, 10)
        edged = cv2.dilate(edged, None, iterations=1)
        edged = cv2.erode(edged, None, iterations=1)
        # find contours in the edge map
        cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        # sort the contours from left-to-right and initialize the
        # 'pixels per metric' calibration variable
        (cnts, _) = contours.sort_contours(cnts)
        pixelsPerMetric = None
        # loop over the contours individually

        # show the output image
        #cv2.imshow("Image", gray)
        #cv2.imshow("Image", edged)
        #cv2.imshow("Image", image)
        #cv2.waitKey(0)
        # loop over the contours individually
        for c in cnts:
            # if the contour is not sufficiently large, ignore it
            if cv2.contourArea(c) < 300:
                continue
            # compute the rotated bounding box of the contour
            orig = frame.copy()
            box = cv2.minAreaRect(c)
            box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
            box = np.array(box, dtype="int")
            # order the points in the contour such that they appear
            # in top-left, top-right, bottom-right, and bottom-left
            # order, then draw the outline of the rotated bounding
            # box
            box = perspective.order_points(box)
            cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)
            # loop over the original points and draw them
            for (x, y) in box:
                cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)
                # unpack the ordered bounding box, then compute the midpoint
                # between the top-left and top-right coordinates, followed by
                # the midpoint between bottom-left and bottom-right coordinates
                (tl, tr, br, bl) = box
                (tltrX, tltrY) = midpoint(tl, tr)
                (blbrX, blbrY) = midpoint(bl, br)
                # compute the midpoint between the top-left and top-right points,
                # followed by the midpoint between the top-righ and bottom-right
                (tlblX, tlblY) = midpoint(tl, bl)
                (trbrX, trbrY) = midpoint(tr, br)
                # draw the midpoints on the image
                cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
                cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
                cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
                cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)
                # draw lines between the midpoints
                cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
                    (255, 0, 255), 2)
                cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
                    (255, 0, 255), 2)
                # compute the Euclidean distance between the midpoints
                dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
                dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
                # if the pixels per metric has not been initialized, then
                # compute it as the ratio of pixels to supplied metric
                # (in this case, inches)
                if pixelsPerMetric is None:
                    pixelsPerMetric = dB / 0.044

                # compute the size of the object
                dimA = dA / pixelsPerMetric
                dimB = dB / pixelsPerMetric
                # draw the object sizes on the image
                cv2.putText(orig, "{:.2f}m".format(dimA),
                    (int(tltrX - 15), int(tltrY + 10)), cv2.FONT_HERSHEY_SIMPLEX,
                    0.65, (255, 255, 255), 2)
                cv2.putText(orig, "{:.2f}m".format(dimB),
                    (int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,
                    0.65, (255, 255, 255), 2)
                # show the output image
                cv2.imshow("Image", orig)
                cv2.waitKey(0)

            height = "{:.2f}".format(dimA)
            print('Height : ', "{:.2f}m".format(dimA))
        return height
   

In [5]:
def detectByPathImage(path):
    image = cv2.imread(path)
    #Take out Backgroung
    image = remove(image)

    image = imutils.resize(image, width = min(800, image.shape[1])) 

    height = detect(image)
    return height
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

In [6]:
def bmicalculculate(filepath):
    height = detectByPathImage(filepath)
    print(height)
    weight = int(input("How much do you weigh: "))
    bmi = weight/(float(height)**2)
    return bmi

In [ ]:
# Create an instance of tkinter frame
win = Tk()

# Set the geometry of tkinter frame
win.geometry("700x350")

def open_file():
    file = filedialog.askopenfile(mode='r', filetypes=[('picture files', '*.jpg')])
    if file:
        filepath = os.path.abspath(file.name) 
        Label(win, text="The File is located at : " + str(filepath), font=('Aerial 11')).pack()
        bmi = bmicalculculate(filepath)
        Label(win, text="The BMI calculated is : " + str(bmi), font=('Aerial 20')).pack()
        
        

# Add a Label widget
label = Label(win, text="Calculate BMI Of Person in Picture", font=('Georgia 13'))
label.pack(pady=10)

# Create a Button
ttk.Button(win, text="Browse", command=open_file).pack(pady=20)

win.mainloop()

Human Detected :  1
Height :  1.70m
Height :  1.83m
1.83
How much do you weigh: 70
Human Detected :  1
Height :  1.65m
Height :  1.75m
1.75
How much do you weigh: 76
